In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, time
import math
# pandas max row 
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [2]:
url = '/content/'
feature_list = ['3시간기온', '강수형태','강수확률','습도', '풍속','풍향','하늘상태']
place = '감천제1동' #부산발전소 위치
period = ['201706_201801','201802_201812','201901_201912','202001_202012','202101_202106']

In [3]:
#merge all csv & using only UTC 17 time
def call_all_csv(feature = '') :
  
  df = pd.read_csv(url+place+'_'+feature+'_'+period[0]+'.csv')
  df.columns = [' format: day', 'hour', 'forecast', 'value']
  #hour : predicting time / foracast : value

  for i in period[1:] :
    
    df01 = pd.read_csv(url+place+'_'+feature+'_'+i+'.csv')
    df01.columns = [' format: day', 'hour', 'forecast', 'value']
    #daytime is in header -> add data to column
    start_name = ' Start : ' + i[0:8]
    df=df.append({' format: day' : start_name} , ignore_index=True)
    df = df.append(df01, ignore_index = True)

  df = df.reset_index(drop=True)
  add_yymm_list(df)
  df = df.astype({'hour' : 'str'})
  df = df[df['hour'] == '1700.0'] #using UTC 17:00, KST 2:00

  df = add_forcasttime(df)
  df = df.drop({' format: day', 'hour', 'forecast','forecast_time', 'forcast_date'},1)

  return df    

In [4]:
def add_yymm_list(dataframe = '') :
    date = dataframe[dataframe['hour'].isnull() == True].copy() #date info row
    date['yymm'] = date[' format: day'].str.slice(start=9, stop=15)
    copy_df = dataframe[' format: day'].copy()
    #display(date)
    index = date.index

    x = 0

    for i in range(len(index)):
      y = index[i]

      if x == 0 :
        x = index[0]
        copy_df[:] = '201706'

      else : 
        date_info = date['yymm'][x]
        copy_df[x:y] = date_info
        x = index[i]
      
      if x == index[-1] :
        copy_df[x:] = '202106'

    dataframe['forecast_time'] = copy_df
    return dataframe[dataframe['hour'].isnull() == False]

In [5]:
def add_forcasttime(dataframe = '') :
  copy_df = dataframe[dataframe['hour'].isnull() == False].copy()
  copy_df[' format: day'] = copy_df[' format: day'].apply(lambda x: "{:0>2d}".format(int(x)))
  copy_df['time'] = copy_df['forecast'] + 2
  copy_df['forcast_date'] = dataframe['forecast_time'] + copy_df[' format: day']
  copy_df['forcast_time'] = pd.to_datetime(copy_df['forcast_date'])

  return copy_df

#기상청데이터

## feature별 묶기
- Forecast time : 예보 발표 시점
- forecast : 예보 시간 (ex - Forecast time:2018-03-01 11:00:00, forecast:4.0 => 2018-03-01 11:00:00에 발표한 2018-03-01 15:00:00 예보
- 강수형태 : 없음 : 0, 비 : 1, 비/눈 : 2, 눈 : 3, 소나기 : 4
- 하늘상태 : 맑음 : 1, 구름많음 : 3, 흐림 : 4

In [6]:
df_weather = pd.DataFrame()

for i in feature_list :
  tmp = call_all_csv(i)
  tmp = tmp[tmp['time'] < 25].copy()

  if i == '3시간기온' :
    df_weather['forcast_time'] = tmp['forcast_time'] + tmp['time'].apply(lambda x : timedelta(hours = x))

  df_weather[i] = tmp['value']

df_weather.head()

,forcast_time,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
90,2017-06-01 06:00:00,18.0,0.0,0.0,80.0,2.1,108.0,1.0
91,2017-06-01 09:00:00,21.0,0.0,0.0,65.0,2.4,111.0,1.0
92,2017-06-01 12:00:00,22.0,0.0,0.0,65.0,2.2,114.0,1.0
93,2017-06-01 15:00:00,20.0,0.0,0.0,60.0,2.9,95.0,1.0
94,2017-06-01 18:00:00,19.0,0.0,0.0,65.0,1.0,86.0,1.0


# 부산태양광

In [7]:
solar = pd.read_csv('/content/solar_201706_202006.csv',encoding='cp949')
solar['년월일'] = solar['년월일'].str.replace('-','')
solar = solar.drop({ '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별, kw)',
       '최소(시간별, kw)'},1)

In [8]:
solar.head()

,년월일,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,20170601,0,0,0,0,0,5.28,23.28,31.44,33.12,50.16,96.72,120.96,118.80,107.76,90.96,62.64,32.88,10.08,1.44,0.0,0,0,0,0
1,20170602,0,0,0,0,0,5.04,31.68,60.48,85.68,122.16,133.44,135.60,127.20,114.96,95.52,65.76,32.16,8.64,0.96,0.0,0,0,0,0
2,20170603,0,0,0,0,0,5.04,24.48,65.52,94.32,113.76,129.60,137.28,124.80,111.36,94.08,65.28,20.40,9.60,1.68,0.0,0,0,0,0
3,20170604,0,0,0,0,0,4.08,30.48,68.16,85.68,119.52,127.92,134.64,128.40,116.40,95.28,66.00,32.16,9.36,0.96,0.0,0,0,0,0
4,20170605,0,0,0,0,0,8.40,30.48,66.00,93.60,114.00,129.84,127.20,120.72,118.80,78.96,65.28,32.64,12.72,2.64,0.0,0,0,0,0


# 데이터종합

In [9]:
start_date = datetime.datetime(2017, 6, 1, 0)
end_date = datetime.datetime(2020, 7, 1, 0)

In [10]:
str_date_list = [] 
while start_date != end_date: 
  
  str_date_list.append(start_date.strftime("%Y%m%d%H")) 
  start_date += timedelta(hours=1)

In [11]:
df_total = pd.DataFrame(index=str_date_list,dtype=float)

## solar

In [12]:
solar_t = solar.transpose()	#행 열 전환
solar_t.rename(columns=solar_t.iloc[0], inplace=True)
solar_t = solar_t.drop(solar_t.index[0])

In [13]:
solar_result = [0]

for i in solar_t.columns:
  for x in range(24) :
    solar_result.append(solar_t[i].values[x])

df_total['solar_result'] = solar_result[:-1]

In [14]:
df_total = df_total.reindex(df_total.columns.tolist() + feature_list, axis=1)
df_total

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
2017060100,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060101,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060102,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060103,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060104,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2020063019,2.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020063020,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020063021,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020063022,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##weather

In [15]:
for i in df_weather.index :
  tmp = df_weather.loc[i]
  datetime = tmp['forcast_time'].strftime("%Y%m%d%H")
  df_total.at[datetime,'3시간기온' : ] = tmp['3시간기온' :]

df_total

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태
2017060100,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060101,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060102,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060103,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017060104,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021063012,NaN,27.0,NaN,30.0,65.0,4.3,56.0,4.0
2021063015,NaN,27.0,NaN,20.0,65.0,4.5,63.0,3.0
2021063018,NaN,26.0,NaN,20.0,70.0,4.9,55.0,3.0
2021063021,NaN,23.0,NaN,0.0,80.0,3.9,42.0,1.0


In [16]:
df_total_inter = df_total.interpolate()

# 태양의 남중고도
(1) interpolate
(2) math

In [17]:
altitude = pd.read_csv('/content/solar_info.csv')
altitude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   altitudeMeridian  1492 non-null   float64
 1   altitude_09       1492 non-null   float64
 2   altitude_12       1492 non-null   float64
 3   altitude_15       1492 non-null   float64
 4   altitude_18       1492 non-null   float64
 5   azimuth_09        1492 non-null   float64
 6   azimuth_12        1492 non-null   float64
 7   azimuth_15        1492 non-null   float64
 8   azimuth_18        1492 non-null   float64
 9   date              1492 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 116.7 KB


In [18]:
time_list = ['09','12','15','18']
df_alti = pd.DataFrame(columns=['date', 'altitude','azimuth'])

for i in time_list :
  tmp = altitude[['date','altitude_'+i,'azimuth_'+i]].copy()
  tmp['date'] = altitude['date'].astype('str') + i
  tmp.columns = ['date', 'altitude','azimuth']
  df_alti = df_alti.append(tmp, ignore_index=True)

df_alti

,date,altitude,azimuth
0,2017060109,44.23,92.42
1,2017060209,44.24,92.31
2,2017060309,44.26,92.20
3,2017060409,44.27,92.10
4,2017060509,44.28,92.00
...,...,...,...
5963,2021062718,18.23,286.00
5964,2021062818,18.24,285.56
5965,2021062918,18.25,285.52
5966,2021063018,18.26,285.47


In [19]:
df_total_inter['date'] = df_total_inter.index
total = pd.merge(df_total_inter, df_alti, on= 'date', how='outer', right_index=True)
total = total[9:27025] #값이 없는 날짜 제외(분석 범위 밖임)
total.head(20)

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태,date,altitude,azimuth
2017060109,33.12,21.000000,0.0,0.000000,65.000000,2.400000,111.000000,1.000000,2017060109,44.23,92.42
2017060110,50.16,21.333333,0.0,0.000000,65.000000,2.333333,112.000000,1.000000,2017060110,NaN,NaN
2017060111,96.72,21.666667,0.0,0.000000,65.000000,2.266667,113.000000,1.000000,2017060111,NaN,NaN
2017060112,120.96,22.000000,0.0,0.000000,65.000000,2.200000,114.000000,1.000000,2017060112,76.04,158.50
2017060113,118.80,21.333333,0.0,0.000000,63.333333,2.433333,107.666667,1.000000,2017060113,NaN,NaN
2017060114,107.76,20.666667,0.0,0.000000,61.666667,2.666667,101.333333,1.000000,2017060114,NaN,NaN
2017060115,90.96,20.000000,0.0,0.000000,60.000000,2.900000,95.000000,1.000000,2017060115,53.08,260.00
2017060116,62.64,19.666667,0.0,0.000000,61.666667,2.266667,92.000000,1.000000,2017060116,NaN,NaN
2017060117,32.88,19.333333,0.0,0.000000,63.333333,1.633333,89.000000,1.000000,2017060117,NaN,NaN
2017060118,10.08,19.000000,0.0,0.000000,65.000000,1.000000,86.000000,1.000000,2017060118,16.44,285.35


In [20]:
#Trial 01 : Interpolate
total['date'] = pd.to_datetime(total['date'], format = '%Y%m%d%H')
total['hour'] = total['date'].apply(lambda x : x.hour).to_list()
trial_1 = total[['hour','azimuth','altitude']].copy()

for i in trial_1.index :
  if (trial_1.loc[i,'hour'] < 6) or (trial_1.loc[i, 'hour'] > 20)  : #밤시간 = 0
    trial_1.loc[i,'azimuth'] = 0
    trial_1.loc[i,'altitude'] = 0

total['altitude'] = trial_1['altitude'].astype('float').interpolate()
total['azimuth'] = trial_1['azimuth'].astype('float').interpolate() 

In [21]:
import datetime
# Trial 02 : Mathmethic
# calculate declination
datetime_list = []
declination = []
a = datetime.datetime(2017, 1, 1, 0)

for i in range(1,371,10) :
  datetime_list.append(a)
  a += timedelta(10)
  dec_v = -23.44*math.cos(math.radians(360/365*(i+10)))
  declination.append(dec_v)

alti_df = pd.DataFrame()
alti_df['time'] = datetime_list
alti_df['declin'] = declination

In [22]:
def cal_elev_azi(decl_v,time_value,hour,coord = 35.6) :
  elevation = math.degrees(math.asin(math.sin(math.radians(decl_v))*math.sin(math.radians(coord))+math.cos(math.radians(decl_v))*math.cos(math.radians(coord))*math.cos(math.radians(time_value))))
  azimuth = math.degrees(math.asin((- math.sin(math.radians(time_value))*math.cos(math.radians(decl_v)))/(math.sin(math.radians(90-elevation)))))

  if hour < 5 : azimuth = azimuth
  elif hour < 18 : azimuth = 180 - azimuth
  else : azimuth = 360 + azimuth

  return elevation, azimuth

In [23]:
decli_total = []
time_total = []
ele_total = []
azi_total = []

for i in total['date']:
  time_value = -180 + i.hour*15
  time_total.append(time_value)
  coord = 35.6

  base_day = datetime.datetime(i.year, 1, 1, 0)
  diff = (i - base_day).days // 10
  decl_v = alti_df.loc[diff, 'declin']

  elevation,azimuth = cal_elev_azi(decl_v,time_value,i.hour)
  decli_total.append(decl_v)
  ele_total.append(elevation)
  azi_total.append(azimuth)

total['altitude_m'] = ele_total
total['azimuth_m'] = azi_total
total.head()

,solar_result,3시간기온,강수형태,강수확률,습도,풍속,풍향,하늘상태,date,altitude,azimuth,hour,altitude_m,azimuth_m
2017060109,33.12,21.000000,0.0,0.0,65.000000,2.400000,111.000000,1.0,2017-06-01 09:00:00,44.230000,92.420000,9,48.617684,96.897301
2017060110,50.16,21.333333,0.0,0.0,65.000000,2.333333,112.000000,1.0,2017-06-01 10:00:00,54.833333,114.446667,10,60.487336,109.600378
2017060111,96.72,21.666667,0.0,0.0,65.000000,2.266667,113.000000,1.0,2017-06-01 11:00:00,65.436667,136.473333,11,71.021213,132.383793
2017060112,120.96,22.000000,0.0,0.0,65.000000,2.200000,114.000000,1.0,2017-06-01 12:00:00,76.040000,158.500000,12,76.252859,180.000000
2017060113,118.80,21.333333,0.0,0.0,63.333333,2.433333,107.666667,1.0,2017-06-01 13:00:00,68.386667,192.333333,13,71.021213,227.616207


In [24]:
! pip install plotly==5.1.0
! pip install cufflinks
! pip install chart_studio
! pip install plotly plotly chart-studio

     |████████████████████████████████| 20.6 MB 52.8 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 64 kB 1.8 MB/s 


In [25]:
import plotly.graph_objects as go

f = go.FigureWidget()
f.add_scatter(y=total['azimuth'][:100],name = 'Interpolate');
f.add_scatter(y=total['azimuth_m'][:100],name = 'Mathmatics');
f.update_layout(plot_bgcolor='white')
f.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
f.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
f.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightGrey')
f.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightGrey')

f.show()

In [26]:
f.write_html('azimuth.html')

In [27]:
total = total.drop(['date','hour'],1)
total.to_csv('total_data.csv')